## 2.1 Contents<a id='2.1_Contents'></a>
* [2 Data wrangling](#2_Data_wrangling)
  * [2.1 Contents](#2.1_Contents)
  * [2.2 Introduction](#2.2_Introduction)
    * [2.2.1 Recap Of Data Science Problem](#2.2.1_Recap_Of_Data_Science_Problem)
    * [2.2.2 Introduction To Notebook](#2.2.2_Introduction_To_Notebook)
  * [2.3 Imports](#2.3_Imports)
  * [2.4 Objectives](#2.4_Objectives)
  * [2.5 Load The Ski Resort Data](#2.5_Load_The_Ski_Resort_Data)
  * [2.6 Explore The Data](#2.6_Explore_The_Data)
    * [2.6.1 Find Your Resort Of Interest](#2.6.1_Find_Your_Resort_Of_Interest)
    * [2.6.2 Number Of Missing Values By Column](#2.6.2_Number_Of_Missing_Values_By_Column)
    * [2.6.3 Categorical Features](#2.6.3_Categorical_Features)
      * [2.6.3.1 Unique Resort Names](#2.6.3.1_Unique_Resort_Names)
      * [2.6.3.2 Region And State](#2.6.3.2_Region_And_State)
      * [2.6.3.3 Number of distinct regions and states](#2.6.3.3_Number_of_distinct_regions_and_states)
      * [2.6.3.4 Distribution Of Resorts By Region And State](#2.6.3.4_Distribution_Of_Resorts_By_Region_And_State)
      * [2.6.3.5 Distribution Of Ticket Price By State](#2.6.3.5_Distribution_Of_Ticket_Price_By_State)
 * [2.7 Summary](#2.7_Summary)

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import sklearn
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import python_weather
import asyncio

from pandas.tseries.holiday import USFederalHolidayCalendar
import holidays  # Assuming you have the 'holidays' library installed

In [4]:
#I was planning to use historic sales data for the project but the product data was too messy so switched directions


# categories_list = pd.read_csv('../data/raw/category-list.csv')
# inconsistent_categories = set(products['Category']).difference(categories_list)
# print(len(inconsistent_categories))

sales_csv = "../data/raw/Sales 1-3-2024.csv"
sales_category_excel = "../data/raw/Sales_By_Category_20231226_1648.xlsx"
sales_history_csv = "../data/raw/Sales/2023 - Sunlight Ski and Bike Shop.csv"

# sales_cat = pd.read_excel(sales_category_excel)
sales = pd.read_csv(sales_history_csv,  encoding='latin-1')

sales.head()


,SKU,Description,Category,Vendor,SaleDate,CustomerLocation,Quantity,Total,Month,Day,Year,MSRP,Discount,Sales,Cost,Profit,Margin
0,4DM73487622R,Smith - 4D MAG Goggle - Connor Ryan Storm Blue...,Accessories - Winter - Goggles,Smith Optics,1/1/2023,WY,1,320.00,1,1,2023,320.00,0.00,320.00,160.00,160.00,50.00
1,541745,Stans - Tubless Tire Sealant - 16oz,Parts - Tubeless Accessories,Quality Bicycle Products,1/1/2023,NaN,1,20.00,1,1,2023,20.00,0.00,20.00,10.45,9.55,47.75
2,541745,Stans - Tubless Tire Sealant - 16oz,Parts - Tubeless Accessories,Trek Bicycle Corporation,1/1/2023,NaN,1,20.00,1,1,2023,20.00,0.00,20.00,10.45,9.55,47.75
3,ADU23584956L,Screamer - Neckwarmer - Charcoal/Sunlight - OSFA,Logo Merchandise - Clothing - Buffs/Facemasks,Screamer,1/1/2023,NaN,1,12.99,1,1,2023,25.98,0.00,25.98,8.84,17.14,65.97
4,ANI11300719M,Black Crows - Anima 115 Skis - 189.2cm,Ski Hardgoods - Skis - Men,Black Crows,1/1/2023,CO,1,538.90,1,1,2023,999.95,461.05,538.90,489.91,48.99,9.09


In [5]:
# sns.stripplot(data=sales,
#             x='Category',
#             y='Month', s=5)
# plt.xlabel('Category')
# plt.ylabel('Month')
# sns.despine()
# # # plt.savefig('casualty breakdown')
# plt.show()

In [6]:
# corrmat = sales.corr()
# f, ax = plt.subplots(figsize=(12, 9))
# hm = sns.heatmap(round(corrmat,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
#                  linewidths=.05)
# f.subplots_adjust(top=0.93)
# t= f.suptitle('Match Data Feature Correlation for Sales', fontsize=14)

In [7]:
#Create a pairplot for all numeric variables
# sns.pairplot(sales, hue='Category', palette='Set1')
# plt.show()

In [8]:
def is_holiday(x): 
    usa_holidays = holidays.UnitedStates()  # Use 'UnitedStates' instead of 'country_holidays('USA')'
    if x in usa_holidays:  # Check if the date is in the list of holidays
        return 1
    else:
        return 0

sales['holidays'] = sales['SaleDate'].apply(is_holiday) 

NameError: name 'weather' is not defined

In [17]:
import asyncio
import nest_asyncio
import pandas as pd
import python_weather

# # help(python_weather)
# # Allow nested use of asyncio.run in Jupyter
# nest_asyncio.apply()

# async def get_weather(city, date):
#     async with python_weather.Client(unit=python_weather.IMPERIAL) as client:
#         weather = await client.get(city)
#         return {'date': date, 'weather_type': weather.current.weather['description']}

# # Assuming you have a DataFrame 'sales' with a 'SaleDate' column
# sales['weather_type'] = None  # Initialize the new column

# # Run the asynchronous tasks
# tasks = [get_weather('Glenwood Springs, CO', date) for date in sales['SaleDate']]
# results = await asyncio.gather(*tasks)

# # Update the DataFrame with weather information
# for result in results:
#     index = sales[sales['SaleDate'] == result['date']].index[0]
#     sales.at[index, 'weather_type'] = result['weather_type']

# # Display the updated DataFrame
# print(sales.head())



import asyncio
from datetime import timedelta
from python_weather import Client

async def get_weather(city, date):
    async with python_weather.Client(unit=python_weather.IMPERIAL) as client:
        weather = await client.get(city, date=date)
        return {'date': date, 'weather_type': weather.current.weather['description']}

# Initialize a new event loop for Jupyter
nest_asyncio.apply()

# Run the asynchronous tasks
tasks = [get_weather('Glenwood Springs, CO', date) for date in sales['SaleDate']]
results = await asyncio.gather(*tasks)

# Merge weather data with sales data
weather_df = pd.DataFrame(results)
sales = pd.merge(sales, weather_df, how='left', on='date')

# Display the updated DataFrame
print(sales.head())

TypeError: Client.get() got an unexpected keyword argument 'date'